In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score

In [2]:
df = pd.read_csv("/home/jupyter/PS_20174392719_1491204439457_log.csv.zip")

In [3]:
def custom_f_score(y_true, y_pred):
    beta = 1.25
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    print('precision: {} \t recall: {}'.format(precision, recall))
    if float(precision + recall) != 0:
        return (1 + beta**2) * (precision * recall) / float(beta**2 * precision + recall)
    else:
        return 0

In [4]:
def evaluate_by_transaction_type(df, trans_type):
    sample_df = df[df['type'] == trans_type].copy()
    
    empty_money_fraud_condition = (sample_df['amount'] == sample_df['oldbalanceOrg'])
    overlimit_fraud_condition = (sample_df['type'] == "TRANSFER") & (sample_df['amount'] == 10000000) & (sample_df['oldbalanceOrg'] >= 10000000)
    
    sample_df.loc[(empty_money_fraud_condition | overlimit_fraud_condition), 'prediction'] = 1
    sample_df['prediction'] = sample_df['prediction'].fillna(0)
    
    f_score = custom_f_score(sample_df['isFraud'], sample_df['prediction'])
    
    return f_score

In [5]:
evaluate_by_transaction_type(df, "TRANSFER")

precision: 1.0 	 recall: 0.9978032706858677


0.9986593816472326

In [6]:
evaluate_by_transaction_type(df, "CASH_OUT")

precision: 1.0 	 recall: 0.9939261418853256


0.9962876285965454

In [7]:
empty_money_fraud_condition = (df['amount'] == df['oldbalanceOrg'])
overlimit_fraud_condition = (df['type'] == "TRANSFER") & (df['amount'] == 10000000) & (df['oldbalanceOrg'] >= 10000000)

In [8]:
print("Number of fraud trans captured: ", df[(empty_money_fraud_condition | overlimit_fraud_condition)].shape[0])
print("Total number of fraud trans in data: ", df[df['isFraud'] == 1]['isFraud'].shape[0])
print("Number of fraud trans escaped: ", df[df['isFraud'] == 1]['isFraud'].shape[0] - df[(empty_money_fraud_condition | overlimit_fraud_condition)].shape[0])
print("Coverage: ", df[(empty_money_fraud_condition | overlimit_fraud_condition)].shape[0] / df[df['isFraud'] == 1]['isFraud'].shape[0])

Number of fraud trans captured:  8179
Total number of fraud trans in data:  8213
Number of fraud trans escaped:  34
Coverage:  0.9958602215999026
